In [21]:
import random
import math
import time
import tracemalloc
import sys

# Set seed for reproducibility
random.seed(42)

# Normalize input data function
def normalize_data(x, min_val, max_val):
    return [(val - min_val) / (max_val - min_val) for val in x]

# Measure memory usage
def get_memory_usage():
    return tracemalloc.get_traced_memory()[1]  # Peak memory usage

# Generate synthetic patient data
def generate_patient_data(num_samples=1000):
    X = [[random.uniform(36, 40),   # Body temperature (°C)
          random.uniform(80, 180),  # Systolic Blood Pressure (mmHg)
          random.uniform(60, 120),  # Diastolic Blood Pressure (mmHg)
          random.uniform(60, 150)]  # Heart Rate (bpm)
         for _ in range(num_samples)]
    X = [normalize_data(x, 36, 180) for x in X]  # Normalize inputs
    y = [classify_health_status(x) for x in X]
    return X, y

# Classify health status based on synthetic thresholds
def classify_health_status(patient):
    temp, sys_bp, dia_bp, heart_rate = patient
    temp = temp * 144 + 36  # Revert normalization for classification
    sys_bp = sys_bp * 144 + 36
    dia_bp = dia_bp * 144 + 36
    heart_rate = heart_rate * 144 + 36

    if temp < 37.5 and 90 <= sys_bp <= 140 and 60 <= dia_bp <= 90 and 60 <= heart_rate <= 100:
        return 0  # Healthy
    elif 37.5 <= temp < 39 or sys_bp < 90 or sys_bp > 160 or dia_bp < 60 or dia_bp > 100 or heart_rate < 50 or heart_rate > 120:
        return 1  # At Risk
    else:
        return 2  # Critical

# Start tracking memory
tracemalloc.start()

# Generate dataset
X_train, y_train = generate_patient_data(1000)
X_test, y_test = generate_patient_data(200)

# Neural Network Parameters
input_size = 4
hidden_size = 16
hidden_layer_2 = 16
output_size = 3
learning_rate = 0.002
epochs = 20

# Initialize Weights and Biases
W1 = [[random.uniform(-1, 1) for _ in range(hidden_size)] for _ in range(input_size)]
b1 = [random.uniform(-1, 1) for _ in range(hidden_size)]
W2 = [[random.uniform(-1, 1) for _ in range(hidden_layer_2)] for _ in range(hidden_size)]
b2 = [random.uniform(-1, 1) for _ in range(hidden_layer_2)]
W3 = [[random.uniform(-1, 1) for _ in range(output_size)] for _ in range(hidden_layer_2)]
b3 = [random.uniform(-1, 1) for _ in range(output_size)]

# Activation Functions
def relu(x):
    return max(0, x)

def softmax(X):
    max_x = max(X) if X else 0  # Prevent empty list issues
    exp_values = [math.exp(x - max_x) for x in X]
    sum_values = sum(exp_values) if sum(exp_values) != 0 else 1  # Prevent division by zero
    return [x / sum_values for x in exp_values]

# Forward Propagation
def forward(x):
    hidden1 = [relu(sum(x[i] * W1[i][j] for i in range(input_size)) + b1[j]) for j in range(hidden_size)]
    hidden2 = [relu(sum(hidden1[j] * W2[j][k] for j in range(hidden_size)) + b2[k]) for k in range(hidden_layer_2)]
    output = [sum(hidden2[k] * W3[k][m] for k in range(hidden_layer_2)) + b3[m] for m in range(output_size)]
    return softmax(output)

# Training Loop
start_time = time.time()
for epoch in range(epochs):
    correct_predictions = 0
    for i in range(len(X_train)):
        x = X_train[i]
        y = y_train[i]
        output = forward(x)
        predicted_status = output.index(max(output))
        if predicted_status == y:
            correct_predictions += 1
        target = [1 if j == y else 0 for j in range(output_size)]
        error = [(output[j] - target[j]) for j in range(output_size)]
        for j in range(min(hidden_size, len(error))):
            for k in range(min(hidden_layer_2, len(error))):
                W2[j][k] -= learning_rate * error[k]
            b2[j] -= learning_rate * error[j]
        for k in range(min(hidden_layer_2, len(error))):
            for m in range(output_size):
                W3[k][m] -= learning_rate * error[m]
            b3[k] -= learning_rate * error[k]
    print(f"Epoch {epoch+1} completed, Training Accuracy: {correct_predictions / len(X_train) * 100:.2f}%")
training_time = time.time() - start_time

# Evaluate final accuracy
correct_predictions = 0
for i in range(len(X_test)):
    x = X_test[i]
    y = y_test[i]
    output = forward(x)
    predicted_status = output.index(max(output))
    if predicted_status == y:
        correct_predictions += 1
final_accuracy = correct_predictions / len(X_test) * 100

# Measure memory and latency
peak_memory = get_memory_usage()

print(f"Training Latency: {training_time:.4f} seconds")
print(f"Peak Memory Usage: {peak_memory / 1024:.2f} KB")
print(f"Final Test Accuracy: {final_accuracy:.2f}%")

# Stop memory tracking
tracemalloc.stop()


Epoch 1 completed, Training Accuracy: 28.90%
Epoch 2 completed, Training Accuracy: 79.10%
Epoch 3 completed, Training Accuracy: 82.70%
Epoch 4 completed, Training Accuracy: 82.70%
Epoch 5 completed, Training Accuracy: 82.70%
Epoch 6 completed, Training Accuracy: 82.70%
Epoch 7 completed, Training Accuracy: 82.70%
Epoch 8 completed, Training Accuracy: 82.70%
Epoch 9 completed, Training Accuracy: 82.70%
Epoch 10 completed, Training Accuracy: 82.70%
Epoch 11 completed, Training Accuracy: 82.70%
Epoch 12 completed, Training Accuracy: 82.70%
Epoch 13 completed, Training Accuracy: 82.70%
Epoch 14 completed, Training Accuracy: 82.70%
Epoch 15 completed, Training Accuracy: 82.70%
Epoch 16 completed, Training Accuracy: 82.70%
Epoch 17 completed, Training Accuracy: 82.70%
Epoch 18 completed, Training Accuracy: 82.70%
Epoch 19 completed, Training Accuracy: 82.70%
Epoch 20 completed, Training Accuracy: 82.70%
Training Latency: 8.6819 seconds
Peak Memory Usage: 375.54 KB
Final Test Accuracy: 76.50%